## 1. 영어 Word2Vec 만들기
파이썬의 gensim 패키지에는 Word2Vec을 지원하고 있어, gensim 패키지를 이용하면 손쉽게 단어를 임베딩 벡터로 변환시킬 수 있습니다. 영어로 된 코퍼스를 다운받아 전처리를 수행하고, 전처리한 데이터를 바탕으로 Word2Vec 작업을 진행하겠습니다.

In [1]:
import re
import urllib.request
import zipfile
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize

### 1) 훈련 데이터 이해하기
훈련 데이터를 다운로드 합니다.

In [2]:
# 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/09.%20Word%20Embedding/dataset/ted_en-20160408.xml", filename="data/word2vec_data/ted_en-20160408.xml")

('data/word2vec_data/ted_en-20160408.xml',
 <http.client.HTTPMessage at 0x7f5a442d3730>)

훈련 데이터 파일은 xml 문법으로 작성되어 있어 자연어를 얻기 위해서는 전처리가 필요합니다. 얻고자 하는 실질적 데이터는 영어문장으로만 구성된 내용을 담고 있는 <content>와 </content> 사이의 내용입니다. 전처리 작업을 통해 xml 문법들은 제거하고, 해당 데이터만 가져와야 합니다. 뿐만 아니라, <content>와 </content> 사이의 내용 중에는 (Laughter)나 (Applause)와 같은 배경음을 나타내는 단어도 등장하는데 이 또한 제거해야 합니다.

```
<file id="1">
  <head>
<url>http://www.ted.com/talks/knut_haanaes_two_reasons_companies_fail_and_how_to_avoid_them</url>
       <pagesize>72832</pagesize>
... xml 문법 중략 ...
<content>
Here are two reasons companies fail: they only do more of the same, or they only do what's new.
To me the real, real solution to quality growth is figuring out the balance between two activities:
... content 내용 중략 ...
To me, the irony about the Facit story is hearing about the Facit engineers, who had bought cheap, small electronic calculators in Japan that they used to double-check their calculators.
(Laughter)
... content 내용 중략 ...
(Applause)
</content>
</file>
<file id="2">
    <head>
<url>http://www.ted.com/talks/lisa_nip_how_humans_could_evolve_to_survive_in_space<url>
... 이하 중략 ...
```

### 2) 훈련 데이터 전처리하기
위 데이터를 위한 전처리 코드는 아래와 같습니다.

In [3]:
targetXML = open('data/word2vec_data/ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.
content_text = re.sub(r'\([^)]*\)', '', parse_text)

# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.
sent_text = sent_tokenize(content_text)

# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = [word_tokenize(sentence) for sentence in normalized_text]

In [4]:
print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 273380


총 샘플의 개수는 약 27만 3천개입니다.

In [5]:
# 샘플 3개만 출력
for line in result[:3]:
    print(line)

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']


상위 3개 문장만 출력해보았는데 토큰화가 수행되었음을 볼 수 있습니다. Word2Vec 모델에 텍스트 데이터를 훈련시킵니다.

### 3) Word2Vec 훈련시키기

In [7]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

# model = Word2Vec(sentences=result, size=100, window=5, min_count=5, workers=4, sg=0)
model = Word2Vec(sentences=result, vector_size=100, window=5, min_count=5, workers=4, sg=0)
 

Word2Vec의 하이퍼파라미터값은 다음과 같습니다.

* size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.   
* window = 컨텍스트 윈도우 크기  
* min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)  
* workers = 학습을 위한 프로세스 수  
* sg = 0은 CBOW, 1은 Skip-gram.  

Word2Vec에 대해서 학습을 진행하였습니다. Word2Vec는 입력한 단어에 대해서 가장 유사한 단어들을 출력하는 model.wv.most_similar을 지원합니다. man과 가장 유사한 단어들은 어떤 단어들일까요?

In [8]:
model_result = model.wv.most_similar("man")
print(model_result)

[('woman', 0.8446374535560608), ('guy', 0.8234966993331909), ('lady', 0.7744773626327515), ('boy', 0.7652905583381653), ('girl', 0.7357457280158997), ('gentleman', 0.7313880324363708), ('kid', 0.7204859852790833), ('soldier', 0.6971893310546875), ('poet', 0.6797811985015869), ('friend', 0.6608587503433228)]


man과 유사한 단어로 woman, guy, boy, lady, girl, gentleman, soldier, kid 등을 출력하는 것을 볼 수 있습니다. Word2Vec를 통해 단어의 유사도를 계산할 수 있게 되었습니다.

### 4) Word2Vec 모델 저장하고 로드하기
공들여 학습한 모델을 언제든 나중에 다시 사용할 수 있도록 컴퓨터 파일로 저장하고 다시 로드해보겠습니다. 이 모델을 가지고 향후 시각화를 진행할 예정이므로 꼭 저장해주세요.

In [10]:
model.wv.save_word2vec_format('model_repo/eng_w2v') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format("model_repo/eng_w2v") # 모델 로드

로드한 모델에 대해서 다시 man과 유사한 단어를 출력해보겠습니다.

In [11]:
model_result = loaded_model.most_similar("man")
print(model_result)

[('woman', 0.8446374535560608), ('guy', 0.8234966993331909), ('lady', 0.7744773626327515), ('boy', 0.7652905583381653), ('girl', 0.7357457280158997), ('gentleman', 0.7313880324363708), ('kid', 0.7204859852790833), ('soldier', 0.6971893310546875), ('poet', 0.6797811985015869), ('friend', 0.6608587503433228)]
